In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyxdf

%matplotlib inline

In [10]:
fpath = 'results\sub-P001\ses-S001\eeg\sub-P001_ses-S001_task-Default_run-002_eeg.xdf'
fpath = 'results\sub-P001\ses-S001\eeg\sub-P001_ses-S001_task-vsync_test_run-001_eeg.xdf'
fpath = 'results\sub-P001\ses-S001\eeg\sub-P001_ses-S001_task-vsync_test_run-002_eeg.xdf'
fpath = 'results\sub-P001\ses-S001\eeg\sub-P001_ses-S001_task-vsync_test_run-005_eeg.xdf'
fpath = 'results\sub-P001\ses-S001\eeg\sub-P001_ses-S001_task-vsync_test_psychopy_run-001_eeg.xdf'
fpath = 'C:/Users/Experiment/Documents/CurrentStudy/sub-P001/ses-S001/eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf'
# fpath = 'C:/Users/Experiment/Documents/CurrentStudy/sub-P001/ses-S001/eeg/sub-P001_ses-S001_task-Default_run-001_eeg_old1.xdf'
streams, header = pyxdf.load_xdf(fpath, dejitter_timestamps=False)

for stream in streams:
    print(stream['info']['name'])

['ScreenSensorStream']
['ScreenSequenceStream']


In [11]:
sequence_stream = [s for s in streams if s['info']['name'][0] == 'ScreenSequenceStream'][0]
sensor_stream = [s for s in streams if s['info']['name'][0] == 'ScreenSensorStream'][0]

In [12]:
sequence_stream['time_stamps'].shape

(989,)

In [4]:
%matplotlib qt
timestamps = sequence_stream['time_stamps']
timeseries = sequence_stream['time_series']
timeseries = np.array(timeseries) == ['up']
timeseries = timeseries.ravel()

timestamps_sensor = sensor_stream['time_stamps']
timeseries_sensor = sensor_stream['time_series']

print(timestamps.shape)
print(timestamps_sensor.shape)
start_time = 10 # Ignore fist 10 seconds
timestamps = timestamps[timestamps > timestamps[0] + 5]
timestamps_sensor = timestamps_sensor[timestamps_sensor > timestamps_sensor[0] + 5]
print(timestamps.shape)
print(timestamps_sensor.shape)

plt.scatter(timestamps, np.ones_like(timestamps), label='expected', marker='o')
plt.scatter(timestamps_sensor, np.ones_like(timestamps_sensor), label='actual', marker='x')
plt.legend()
plt.show()

(989,)
(2568,)
(690,)
(1776,)


In [27]:
# print(timestamps.shape)
# print(timestamps_sensor.shape)
# timestamps = timestamps[timestamps > 2090]
# timestamps_sensor = timestamps_sensor[timestamps_sensor > 2090]
# print(timestamps.shape)
# print(timestamps_sensor.shape)

In [28]:
min_diff = 0.002
filtered = [timestamps_sensor[0]]

# Iterate and filter out timestamps_sensor that are too close
for ts in timestamps_sensor[1:]:
    if ts - filtered[-1] >= min_diff:
        filtered.append(ts)

# Convert back to NumPy array
merged_timestamps = np.array(filtered)

print("Original length:", len(timestamps_sensor))
print("Filtered length:", len(merged_timestamps))


Original length: 1776
Filtered length: 704


In [31]:
plt.scatter(timestamps, np.ones_like(timestamps), label='expected', marker='o')
plt.scatter(merged_timestamps, np.ones_like(merged_timestamps), label='actual', marker='x')
plt.legend()
plt.show()

In [32]:
merged_timestamps = merged_timestamps[merged_timestamps > timestamps[0]]
merged_timestamps = merged_timestamps[merged_timestamps < timestamps[-1]]
timestamps = timestamps[:-1]

In [33]:
print(merged_timestamps.shape)
print(timestamps.shape)

(689,)
(689,)


In [35]:
sns.boxplot((merged_timestamps - timestamps))

<Axes: >